# Patch Classification Training

### Part 1: Find all the patches in the small training set of Camelyon16

In [1]:
import repath.data.datasets.camelyon16 as camelyon16
from repath.preprocess.patching import GridPatchFinder
from repath.preprocess.patching import SlidesIndex
from repath.preprocess.tissue_detection import TissueDetectorGreyScale

# index all the patches for the camelyon16 dataset
train_data16 = camelyon16.training()
tissue_detector = TissueDetectorGreyScale()
patch_finder = GridPatchFinder(7, 0, 256, 256)
train_patches16 = SlidesIndex.index_dataset(train_data16, tissue_detector, patch_finder)
train_patches16.summary()

indexing tumor_001.tif
indexing tumor_002.tif
indexing tumor_003.tif
indexing tumor_004.tif
indexing tumor_005.tif
indexing tumor_006.tif
indexing tumor_007.tif
indexing tumor_008.tif
indexing tumor_009.tif
indexing tumor_010.tif
indexing tumor_011.tif
indexing tumor_012.tif
indexing tumor_013.tif
indexing tumor_014.tif
indexing tumor_015.tif
indexing tumor_016.tif
indexing tumor_017.tif
indexing tumor_018.tif
indexing tumor_019.tif
indexing tumor_020.tif
indexing tumor_021.tif
indexing tumor_022.tif
indexing tumor_023.tif
indexing tumor_024.tif
indexing tumor_025.tif
indexing tumor_026.tif
indexing tumor_027.tif
indexing tumor_028.tif
indexing tumor_029.tif
indexing tumor_030.tif
indexing tumor_031.tif
indexing tumor_032.tif
indexing tumor_033.tif
indexing tumor_034.tif
indexing tumor_035.tif
indexing tumor_036.tif
indexing tumor_037.tif
indexing tumor_038.tif
indexing tumor_039.tif
indexing tumor_040.tif
indexing tumor_041.tif
indexing tumor_042.tif
indexing tumor_043.tif
indexing tu

,slide_path,slide_label,background,normal,tumor
0,tumor/tumor_001.tif,tumor,0.0,41636,238
1,tumor/tumor_002.tif,tumor,0.0,18913,27
2,tumor/tumor_003.tif,tumor,0.0,33296,174
3,tumor/tumor_004.tif,tumor,0.0,48162,547
4,tumor/tumor_005.tif,tumor,0.0,18592,71
...,...,...,...,...,...
265,normal/normal_156.tif,normal,0.0,40036,0
266,normal/normal_157.tif,normal,0.0,50905,0
267,normal/normal_158.tif,normal,0.0,53516,0
268,normal/normal_159.tif,normal,0.0,54722,0


In [2]:
from repath.preprocess.sampling import split_camelyon16
from repath.utils.paths import project_root

experiment_name = "example"
experiment_root = project_root() / "experiments" /  experiment_name

train16, valid16 = split_camelyon16(train_patches16, 0.7)
train16.save(experiment_root / "train16_index")
valid16.save(experiment_root / "valid16_index")
for sl in train16:
    print("train", sl.slide_idx)
    
for sl in valid16:
    print("valid", sl.slide_idx)

Splitting Camelyon 16
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_050.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_092.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_017.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_072.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_004.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_052.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_068.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_031.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_110.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_106.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/normal/normal_062.csv
Saving /home/ubuntu/repath/experiments/example/train16_index/norm

In [3]:
project_root()

PosixPath('/home/ubuntu/repath')

In [4]:
from repath.preprocess.sampling import balanced_sample, weighted_random

train_samples = balanced_sample([train16], 5000, sampling_policy=weighted_random)

In [ ]:
from repath.preprocess.patching.patch_index import CombinedIndex, CombinedPatchSet, SlidesIndex, SlidePatchSet
import numpy as np

indexes = [train16]
num_samples = 2800
floor_samples = 1000

index = CombinedIndex.for_slide_indexes(indexes)

# work out how many of each type of patches you have in the index
labels = np.unique(index.patches_df.label)
sum_totals = [np.sum(index.patches_df.label == label) for label in labels]

# find the count for the class that has the lowest count, so we have balanced classes
n_patches = min(sum_totals)

# limit the count for each class to the number of samples we want
n_patches = min(n_patches, num_samples)

# make sure that have a minimun number of samples for each class if available
# classes with smaller that floor with remain the same
n_patches = max(n_patches, floor_samples)
sum_totals = np.minimum(sum_totals, n_patches)  # cap the number sample from each class to n_patches

labels

In [ ]:
import pandas as pd

sampled_patches = pd.DataFrame(columns=index.patches_df.columns)

idx = 1
label = labels[idx]

class_df = index.patches_df[index.patches_df.label == label]

class_df

In [ ]:
class_df['slide_idx'].value_counts()

In [ ]:
#class_df.loc[:, 'freq'] = class_df.groupby('slide_idx')['slide_idx'].transform('count')
freq_vals = class_df.groupby('slide_idx')['slide_idx'].transform('count')
#class_df.loc[:, 'freq'] = freq_vals
#class_df[class_df.slide_idx==75]

In [ ]:
freq_vals = class_df.groupby('slide_idx')['slide_idx'].transform('count').tolist()


In [ ]:
#class_df['freq'] = freq_vals
class_df = class_df.assign(freq=class_df.groupby('slide_idx')['slide_idx'].transform('count').tolist())

In [ ]:
class_df.loc[:, 'freq'] = 1
class_df

In [ ]:
class_df['weights'] = np.divide(1, class_df['freq'])
class_sample = class_df.sample(n=sum_totals[idx], axis=0, replace=False, weights=class_df.weights)
print(sum_totals[idx])
print(class_sample.shape)
class_sample['slide_idx'].value_counts()

In [ ]:
class_sample = class_df.sample(n=sum_totals[idx], axis=0, replace=False)
class_sample['slide_idx'].value_counts()

In [ ]:
from repath.preprocess.sampling import weighted_sample

# do the train validate split
train_samples = weighted_sample([train16], 2800)
valid_samples = weighted_sample([valid16], 1200)

import numpy as np
np.unique(train16.as_combined().patches_df.label)

In [ ]:
from repath.preprocess.sampling import select_annotated

train17 = select_annotated(train17)
valid17 = select_annotated(valid17)

In [ ]:
import repath.data.datasets.camelyon16 as camelyon16
from repath.preprocess.patching import GridPatchFinder
from repath.preprocess.patching import SlidesIndex
from repath.preprocess.tissue_detection import TissueDetectorOTSU

# index all the patches for the camelyon16 dataset
train_data16 = camelyon16.training_small()
tissue_detector = TissueDetectorOTSU()
patch_finder = GridPatchFinder(7, 0, 256, 256)
train_patches16 = SlidesIndex.index_dataset(train_data16, tissue_detector, patch_finder)
train_patches16.summary()

In [ ]:
from repath.preprocess.sampling import split_camelyon16

train16, valid16 = split_camelyon16(train_patches16, 0.7)
train16.save(experiment_root / "train16_index")
valid16.save(experiment_root / "valid16_index")

In [ ]:
from repath.preprocess.sampling import balanced_sample

# do the train validate split
train_samples = balanced_sample([train16, train17], 2800)
valid_samples = balanced_sample([valid16, valid17], 1200)

import numpy as np
np.unique(train17.as_combined().patches_df.label)

In [ ]:
# save out all the patches
train_samples.save_patches(experiment_root / "training_patches")
valid_samples.save_patches(experiment_root / "validation_patches")

## Part 2: Train patch classifier